In [ ]:
import os

import numpy as np
from IPython.display import Video, clear_output

import h5py


def get_label(ans):
    try:
        ans = int(ans)
    except:
        print("Wrong Input")
        return False
    if ans not in [0, 1, 2, 3]:
        print("Invalid option.")
        return False
    if ans == 1:
        return [0]
    elif ans == 2:
        return [1]
    elif ans == 3:
        return [0.5]
    else:
        return [-1]


def create_human_label(video_dir, num_query=100, start_idx=0, width=1000, height=500):
    print("START!")
    interval = range(start_idx, num_query)
    pref_labels = []
    end_labeling = False
    for i in interval:
        label = False
        clear_output(wait=True)
        while not label:
            print(f"\nVideo {i}")
            video_file = os.path.join(video_dir, f"idx{i}.mp4")
            display(
                Video(
                    video_file,
                    width=width,
                    height=height,
                    html_attributes="loop autoplay",
                )
            )
            reward = input(
                f"[{i}/{num_query - 1}] Put Preference (1 (left), 2 (right), 3 (equal)) or 0 to Stop (No Label Given):  "
            ).strip()
            label = get_label(reward)
            if label:
                if label[0] < 0:
                    end_labeling = True
                    break
                pref_labels.append(label[0])
        if end_labeling:
            break

    return pref_labels, start_idx

In [ ]:
num_query = 1000
start_idx = 0
env_pref_file = "../pen_labels/AdroitHandPen-v1_pref.hdf5"
with h5py.File(env_pref_file, "a") as f:
    p_labels, start_idx = create_human_label(
        "../pen_labels/AdroitHandPen-v1", num_query=num_query, start_idx=start_idx
    )
    if "labels" in f:
        if start_idx == 0:
            del f["labels"]
            labels = np.ones(num_query) * -1
            for i, p in enumerate(p_labels):
                labels[i] = p
        else:
            assert start_idx > 0
            labels = f["labels"][:]
            del f["labels"]
            for i, p in enumerate(p_labels):
                labels[start_idx + i] = p
    else:
        assert start_idx == 0
        labels = np.ones(num_query) * -1
        for i, p in enumerate(p_labels):
            labels[i] = p

    f.create_dataset("labels", data=labels, chunks=True)